In [1]:
import torch
import pickle
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
# workaround for torch hub 403 error
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True

/home/yujia/miniconda3/envs/incivility/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# print(torch.hub.list('pytorch/fairseq')) 

In [ ]:
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en',  checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')
de2en.cuda()

In [ ]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de',  checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')
en2de.cuda()

In [51]:
annotated_data_dir = './data/labeled/final_annotated_data_incivility_3030_processed.pickle'
annotated_df = pd.read_pickle(annotated_data_dir)
annotated_df.head()

,id,comment,parent_comment,original_post,divisiveness,divisiveness_keyword,human_incivility,stereotype,namecalling,aspersion,...,bert,attention,type_id,personal_attack,third_party_attack,civility,comment_de,comment_en,bert_aug,attention_aug
0,fysijl2,"If I wanted to watch a stable genius, I'd rath...",It's going to devolve into a campaign stump sp...,Today President Trump is expected to hold his ...,1,"Trump, coronavirus",2,0,3,0,...,"[101, 2065, 1045, 2359, 2000, 3422, 1037, 6540...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,4,"Wenn ich ein stabiles Genie sehen wollte, würd...","If I wanted to see a stable genius, I'd rather...","[101, 2065, 1045, 2359, 2000, 2156, 1037, 6540...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,fygrqc9,If they're willing to shoot a deceptive video ...,"Look, you're simply wrong or haven't done the ...",Media are just plain lying about Kayleigh McEn...,1,Kayleigh McEnany,0,0,0,0,...,"[101, 2065, 2027, 1005, 2128, 5627, 2000, 5607...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,"Wenn sie bereit sind, ein irreführendes Video ...",If they are willing to make a misleading video...,"[101, 2065, 2027, 2024, 5627, 2000, 2191, 1037...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,g0ql4qq,Would be thrilling if he ended up actually bei...,Second challenge is disputing just about every...,Today President Trump is expected to hold his ...,1,Kanye West,1,0,0,0,...,"[101, 2052, 2022, 26162, 2065, 2002, 3092, 203...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,1,"Wäre spannend, wenn er am Ende tatsächlich für...",It would be exciting if he was actually punish...,"[101, 2009, 2052, 2022, 10990, 2065, 2002, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,fz1nlup,Nope they like to let one vote side against th...,LETS GOOOO. Romney found his balls!!,Romney reportedly will vote against controvers...,1,"Romney, controversial Fed nominee",0,0,0,0,...,"[101, 16780, 2027, 2066, 2000, 2292, 2028, 378...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,"Nein, sie lassen gerne eine Seite gegen die Di...","No, they like to let one side vote against thi...","[101, 2053, 1010, 2027, 2066, 2000, 2292, 2028...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,fzd1ea2,"Most conservatives agree with you. However, co...",you know you can commit a violent crime and no...,If african americans commit 3x more violent cr...,1,"african american crime, police",2,1,0,1,...,"[101, 2087, 11992, 5993, 2007, 2017, 1012, 217...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,3,Die meisten Konservativen stimmen Ihnen zu. Ab...,"Most conservatives agree with you, but to comm...","[101, 2087, 11992, 5993, 2007, 2017, 1010, 202...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [53]:
len(annotated_df.bert_aug.values[0])

207

In [ ]:
comments = annotated_df.comment_de.values
# comments[690] = comments[690][:5000]

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
english = []
for i in range(0, len(comments), 10):
    print(i)
    temp = comments[i: i+10]
    english.extend(de2en.translate(temp))
len(english)

In [2]:
annotated_df['comment_en'] = english
annotated_df.to_pickle('./data/labeled/final_annotated_data_incivility_3030_processed_de_en.pickle')

NameError: name 'english' is not defined

In [50]:
roberta_data_dir = './data/labeled/final_annotated_data_roberta.pickle'
df = pd.read_pickle(roberta_data_dir)
df.head()

,id,comment,parent_comment,original_post,divisiveness,divisiveness_keyword,human_incivility,stereotype,namecalling,aspersion,...,vulgarity,other,personal_attack,third_party_attack,civility,roberta,attention_mask,comment_en,roberta_aug,attention_mask_aug
0,fysijl2,"If I wanted to watch a stable genius, I'd rath...",It's going to devolve into a campaign stump sp...,Today President Trump is expected to hold his ...,1,"Trump, coronavirus",2,0,3,0,...,1,0,0,1,4,"[0, 1106, 38, 770, 7, 1183, 10, 4375, 16333, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","If I wanted to see a stable genius, I'd rather...","[0, 1106, 38, 770, 7, 192, 10, 4375, 16333, 6,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,fygrqc9,If they're willing to shoot a deceptive video ...,"Look, you're simply wrong or haven't done the ...",Media are just plain lying about Kayleigh McEn...,1,Kayleigh McEnany,0,0,0,0,...,0,1,0,0,1,"[0, 1106, 51, 214, 2882, 7, 4511, 10, 31405, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",If they are willing to make a misleading video...,"[0, 1106, 51, 32, 2882, 7, 146, 10, 12030, 569...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,g0ql4qq,Would be thrilling if he ended up actually bei...,Second challenge is disputing just about every...,Today President Trump is expected to hold his ...,1,Kanye West,1,0,0,0,...,1,0,0,0,1,"[0, 29042, 28, 16208, 114, 37, 1249, 62, 888, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",It would be exciting if he was actually punish...,"[0, 243, 74, 28, 3571, 114, 37, 21, 888, 14459...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,fz1nlup,Nope they like to let one vote side against th...,LETS GOOOO. Romney found his balls!!,Romney reportedly will vote against controvers...,1,"Romney, controversial Fed nominee",0,0,0,0,...,0,0,0,0,0,"[0, 487, 9877, 51, 101, 7, 905, 65, 900, 526, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","No, they like to let one side vote against thi...","[0, 3084, 6, 51, 101, 7, 905, 65, 526, 900, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,fzd1ea2,"Most conservatives agree with you. However, co...",you know you can commit a violent crime and no...,If african americans commit 3x more violent cr...,1,"african american crime, police",2,1,0,1,...,1,0,0,0,3,"[0, 2895, 9930, 2854, 19, 47, 4, 635, 6, 11389...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","Most conservatives agree with you, but to comm...","[0, 2895, 9930, 2854, 19, 47, 6, 53, 7, 6225, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
df['comment_en'] = annotated_df['comment_en']
df.to_pickle(roberta_data_dir)
df